In [49]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
json_prepared_filtered = dataiku.Dataset("Json_prepared_filtered")
df = json_prepared_filtered.get_dataframe()

In [50]:
import math

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import Sequence
from datetime import timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import numpy as np
import pandas as pd
import time

import os

In [51]:
del df['Timestamp']
df_cpy  = df.set_index('date')

In [52]:
val = df_cpy
scalers={}
for i in df_cpy.columns:
    scaler = MinMaxScaler(feature_range=(-1,1))
    s_s = scaler.fit_transform(val[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+ i] = scaler
    val[i]=s_s

In [53]:
val_to_pred = val.values
val_to_pred = val_to_pred.reshape((1, val_to_pred.shape[0],val_to_pred.shape[1]))

In [54]:
handle = dataiku.Folder("models")
json_config = handle.read_json("model_json")
model = keras.models.model_from_json(json_config)

/opt/dataiku/code-env/lib64/python3.6/site-packages/tensorflow_core/python/framework/tensor_util.py:521: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  tensor_proto.tensor_content = nparray.tostring()


In [55]:
val_pred = model.predict(val_to_pred)

In [57]:
for index,i in enumerate(df_cpy.columns):
    scaler = scalers['scaler_'+i]
    val_pred[:,:,index]=scaler.inverse_transform(val_pred[:,:,index])

In [58]:
val = val_pred.squeeze()
val.shape

(5, 30)

In [60]:
pd.DataFrame(val, columns=df_cpy.columns)

,broker-102/,broker-104/,broker-106/,broker-108/,broker-110/,zook-102/,zook-104/,broker-102/data,broker-104/data,broker-106/data,broker-108/data,broker-110/data,zook-102/data,zook-104/data,broker-101/,broker-103/,broker-105/,broker-107/,broker-109/,zook-101/,zook-103/,zook-105/,broker-101/data,broker-103/data,broker-105/data,broker-107/data,broker-109/data,zook-101/data,zook-103/data,zook-105/data
0,13.829904,12.370465,13.398085,11.131040,5.384354,3.470438,45.658268,77.798096,73.659523,67.849602,34.353062,44.715580,8.136435,6.526124,10.945130,11.969110,8.464610,7.369499,5.603982,3.266891,38.822983,33.399689,74.625633,70.949013,75.151627,28.887962,37.118645,0.237391,0.122937,0.088851
1,13.827413,12.368169,13.395205,11.122213,5.244454,3.469371,44.918064,77.869888,73.728638,67.845085,32.283901,43.195049,7.977165,6.412914,10.929482,12.085797,8.422974,7.560489,5.572600,3.270096,40.814400,33.572910,74.111061,70.940231,75.181366,31.458179,37.625420,0.235559,0.128693,0.098169
2,13.825769,12.367417,13.392929,11.116161,5.169658,3.468504,44.285770,77.907990,73.784630,67.857262,30.998426,42.067047,7.817830,6.364792,10.937129,12.160419,8.395222,7.729565,5.546916,3.274801,42.315041,34.077682,73.630211,70.946190,75.221184,33.471870,37.793404,0.234155,0.131835,0.105052
3,13.824516,12.367314,13.391109,11.112173,5.134713,3.467815,43.666962,77.924744,73.830345,67.873062,30.204741,41.236897,7.668516,6.355106,10.959486,12.206495,8.376280,7.867665,5.524976,3.279732,43.336624,34.676380,73.211441,70.957001,75.236755,34.989853,37.840431,0.233119,0.133609,0.110116
4,13.823488,12.367464,13.389660,11.109614,5.121978,3.467275,43.051113,77.931969,73.866348,67.890060,29.716938,40.646709,7.539484,6.364955,10.988175,12.234900,8.362979,7.974416,5.507166,3.284281,44.010052,35.257278,72.863976,70.969933,75.236031,36.092461,37.887993,0.232376,0.134693,0.113803


In [40]:
lastdate = df["date"].max()
print(lastdate)
for i in range(len(val)):
    lastdate = lastdate + timedelta(minutes=15)
    row = dict()
    
    row["date"] = lastdate
    
    df.append({})

2021-08-05 11:45:00+00:00


TypeError: append() missing 1 required positional argument: 'other'

In [0]:
# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

last_prediction_df = json_prepared_filtered_df # For this sample code, simply copy input to output


# Write recipe outputs
last_prediction = dataiku.Dataset("last_prediction")
last_prediction.write_with_schema(last_prediction_df)